In [4]:
import pygame
from settings import Settings
import game_functions as gf
import agent as ag
import keyboard as k

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from torch.utils.tensorboard import SummaryWriter

LOAD_WEIGHTS = False
SCREEN_INIT = True
# Training epochs for no_gui mode, when (SCREEN_INIT==False)
EPOCHS = 3000

settings = Settings()
print(settings.field_size)
snake,apple = gf.new_game(settings)
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
print(device)
policy_net, target_net, optimizer, memory = ag.create_agent(settings,device,ag.MEMORY_SIZE)
writer = SummaryWriter()

#Optional loading weights of NN from file
if LOAD_WEIGHTS:
    checkpoint = torch.load('.\w_100f_512n_512n', map_location=torch.device('cpu'))
    policy_net.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

(300, 300)
cpu


In [5]:
%%time
run=True
if SCREEN_INIT:
    screen = gf.run_window(settings)
    gf.update_screen(screen,settings,snake,apple)
epoch=0
while run:
    if EPOCHS == str(epoch):
        run = False
    epoch+=1
    state = ag.get_screen(settings, snake, apple)
    
    keys = pygame.key.get_pressed()
    if not keys[pygame.K_LCTRL] and not keys[pygame.K_RCTRL]:
        if choice == 'y':
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    run=False
        action = ag.select_action(state,policy_net)

    elif choice == 'y':
        press=0
        while press==0 and run:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    run=False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_RIGHT:
                        action=torch.tensor([[0]])
                        press=1
                    elif event.key == pygame.K_LEFT:
                        action=torch.tensor([[1]])
                        press=1
                    elif event.key == pygame.K_UP:
                        action=torch.tensor([[2]])
                        press=1
                    elif event.key == pygame.K_DOWN:
                        action=torch.tensor([[3]])
                        press=1
    
    if run:
        gf.check_events(settings,snake,apple,action)
        reward = torch.tensor([snake.reward],device=device)
        if snake.dead:
            writer.add_scalar('Length',len(snake.body),epoch)
            snake,apple = gf.new_game(settings)
        if choice == 'y':
            gf.update_screen(screen,settings,snake,apple)
        next_state = ag.get_screen(settings, snake, apple)
        memory.push(state, action, next_state, reward)
        loss1 = ag.optimize_model(memory,device,policy_net,target_net,optimizer,snake)
        if loss1 != None:
            writer.add_scalar('Loss',loss1,epoch)

error: Window is too large.

In [5]:
print('Save NN weights in file?\n y or n')
choice = input()

if choice =='y':
    torch.save({
                'model_state_dict': policy_net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),

                }, 'w_100f_512n_512n')

Save NN weights in file?
 y or n
n
